In [2]:
import numpy as np
import pickle
import pandas as pd
import json
import os
import copy
import gc
import math
import scipy.sparse as sp

In [3]:
with open('arrays/plsss.pkl','rb') as f:
    mappings = pickle.load(f)
with open('arrays/user_handle.pkl','rb') as f:
    user_handle = pickle.load(f)
with open('arrays/user_id.pkl','rb') as f:
    user_id = pickle.load(f)
with open('arrays/problem_mappings.pkl','rb') as f:
    problem_id = pickle.load(f)
with open('arrays/problem_name.pkl','rb') as f:
    problem_name = pickle.load(f)

In [4]:
problem_AC = np.load('arrays/problem_AC.npy')
problem_attempted = np.load('arrays/problem_attempted.npy')
user_AC = np.load('arrays/user_AC.npy')
user_attempted = np.load('arrays/user_attempted.npy')
mean_user_rating = np.load('arrays/mean_user_rating.npy')

In [5]:
user_problem_data = sp.load_npz('arrays/user_prob_matrix_coo.npz')

In [6]:
print(user_AC[0:100])

[ 1076.   633.   193.  1055.   669.   526.   960.  1415.   934.   922.
   744.   421.   472.   476.  1119.  1175.   884.   152.   614.   220.
  1313.  1253.   694.  1559.   406.   280.  1665.  1006.   792.   861.
   714.   526.   692.   905.  1010.   632.   164.   904.   331.   457.
   420.   308.   790.   967.   288.   268.   218.   576.  1974.  3001.
   454.    48.   896.   774.   856.   244.   625.   881.   406.   164.
   131.   701.   565.  2662.    96.   301.  1197.   227.   707.   778.
   530.   599.   691.   766.   396.   254.    83.   629.   312.   810.
  1492.   634.    43.  1051.   774.    57.  1183.   333.    75.   675.
   476.   325.  1039.   631.   285.   175.   655.  1438.   111.   486.]


In [7]:
print(user_id['tourist'])

0


In [8]:
print(problem_name[100])

22E


In [9]:
def get_problem_id(cid, idx):
    if str(cid)+idx in problem_id:
        return problem_id[str(cid)+idx]
    if cid>len(mappings) or len(idx)>1:
        return -1
    if mappings[cid] !=  0:
        #print(str(cid)+idx)
        idx = chr(int(ord(idx) - mappings[cid][1]))
        cid = cid + mappings[cid][0]
        #print('haha'+str(cid)+idx)
        if str(cid) + idx in problem_id:
            return problem_id[str(cid)+idx]
    return -1

In [10]:
dt = np.dtype({'names':('last_sub', 'first_sub','first_AC_sub','user_problem_AC','user_problem_attempted'),
                          'formats':(np.int64, np.int64,np.int64,np.int32,np.int32)})

In [13]:
x = []
y = []
p_val = []
c_val = []

In [14]:
for i,j,v in zip(user_problem_data.row, user_problem_data.col, user_problem_data.data):
    x.append(i)
    y.append(j)
    c_val.append(v['user_problem_attempted'])
    if v['user_problem_AC'] > 0:
        p_val.append(1)
    else:
        p_val.append(0)

In [17]:
preference = sp.coo_matrix((p_val, (x, y)), shape=(147000,4434))
confidence = sp.coo_matrix((c_val, (x, y)), shape=(147000,4434))

In [19]:
del p_val,c_val

In [21]:
preference = preference.tocsr()
confidence = confidence.tocsr()

In [22]:
print(preference)

  (0, 0)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	0
  (0, 16)	1
  (0, 29)	1
  (0, 30)	1
  (0, 31)	1
  (0, 33)	1
  (0, 39)	1
  (0, 40)	1
  (0, 41)	1
  (0, 43)	0
  (0, 54)	1
  (0, 55)	1
  (0, 56)	1
  (0, 57)	0
  (0, 64)	1
  (0, 65)	1
  (0, 66)	1
  (0, 67)	1
  (0, 68)	1
  (0, 69)	1
  (0, 70)	1
  (0, 71)	1
  :	:
  (146975, 3034)	1
  (146975, 3035)	0
  (146975, 3040)	1
  (146975, 3045)	0
  (146975, 3049)	1
  (146975, 3057)	1
  (146975, 3062)	0
  (146975, 3069)	0
  (146975, 3070)	0
  (146975, 3118)	0
  (146975, 3119)	1
  (146975, 3120)	0
  (146975, 3124)	0
  (146975, 3125)	0
  (146975, 3138)	0
  (146975, 3139)	0
  (146975, 3145)	1
  (146975, 3146)	0
  (146975, 3190)	0
  (146975, 3830)	1
  (146975, 3832)	0
  (146975, 3833)	0
  (146975, 3866)	0
  (146975, 3885)	0
  (146975, 4225)	0


In [23]:
sp.save_npz('pref.npz',preference)
sp.save_npz('confi.npz',confidence)